In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Lambda
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Load and Define Data

In [2]:
df = pd.read_csv('../Data/2010-1-PUMA100_Treated.csv')
df

,AGEP,HINCP,NP,SEX,WIF,HUPAC,HHT,RAC1P,ESR,PUMA,ST
0,48,5,5,2,4,2,1,1,1,100,24
1,45,5,5,1,4,2,1,1,1,100,24
2,16,5,5,1,4,2,1,1,6,100,24
3,14,5,5,1,4,2,1,1,0,100,24
4,17,5,5,1,4,2,1,1,6,100,24
...,...,...,...,...,...,...,...,...,...,...,...
1023,81,2,2,1,1,4,1,1,6,100,24
1024,76,2,2,2,1,4,1,1,6,100,24
1025,66,2,3,1,3,4,1,1,1,100,24
1026,55,2,3,2,3,4,1,1,1,100,24


In [3]:
df=df.fillna(df.mean())
df

,AGEP,HINCP,NP,SEX,WIF,HUPAC,HHT,RAC1P,ESR,PUMA,ST
0,48,5,5,2,4,2,1,1,1,100,24
1,45,5,5,1,4,2,1,1,1,100,24
2,16,5,5,1,4,2,1,1,6,100,24
3,14,5,5,1,4,2,1,1,0,100,24
4,17,5,5,1,4,2,1,1,6,100,24
...,...,...,...,...,...,...,...,...,...,...,...
1023,81,2,2,1,1,4,1,1,6,100,24
1024,76,2,2,2,1,4,1,1,6,100,24
1025,66,2,3,1,3,4,1,1,1,100,24
1026,55,2,3,2,3,4,1,1,1,100,24


In [4]:
df=np.round(df)
df

,AGEP,HINCP,NP,SEX,WIF,HUPAC,HHT,RAC1P,ESR,PUMA,ST
0,48,5,5,2,4,2,1,1,1,100,24
1,45,5,5,1,4,2,1,1,1,100,24
2,16,5,5,1,4,2,1,1,6,100,24
3,14,5,5,1,4,2,1,1,0,100,24
4,17,5,5,1,4,2,1,1,6,100,24
...,...,...,...,...,...,...,...,...,...,...,...
1023,81,2,2,1,1,4,1,1,6,100,24
1024,76,2,2,2,1,4,1,1,6,100,24
1025,66,2,3,1,3,4,1,1,1,100,24
1026,55,2,3,2,3,4,1,1,1,100,24


In [5]:
df.shape

(1028, 11)

# Data Preprocessing

In [6]:
X_train,X_test=train_test_split(df,test_size=200)


In [7]:
X_train

,AGEP,HINCP,NP,SEX,WIF,HUPAC,HHT,RAC1P,ESR,PUMA,ST
1001,52,2,2,1,1,4,1,1,6,100,24
885,14,5,4,1,3,2,1,1,0,100,24
489,20,4,4,2,4,4,1,1,6,100,24
451,60,2,2,2,2,4,1,1,6,100,24
886,11,5,4,1,3,2,1,1,0,100,24
...,...,...,...,...,...,...,...,...,...,...,...
24,7,4,3,1,3,2,1,1,0,100,24
118,17,4,4,2,4,2,1,1,1,100,24
521,5,2,5,1,1,3,1,1,0,100,24
864,28,2,6,2,2,1,2,1,6,100,24


In [8]:
minmax=MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.fit_transform(X_test)

In [9]:
X_train

array([[0.55319149, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.14893617, 0.75      , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.21276596, 0.5       , 0.4       , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.05319149, 0.        , 0.6       , ..., 0.        , 0.        ,
        0.        ],
       [0.29787234, 0.        , 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.55319149, 0.5       , 0.        , ..., 0.16666667, 0.        ,
        0.        ]])

In [10]:
def fit_batchsize(X,batch_size):
    n_size = (len(X)//batch_size)*batch_size
    X = X[0:n_size]

    return X
batch_size = 10
X_train = fit_batchsize(X_train, batch_size)
X_test = fit_batchsize(X_test, batch_size)

# Define VAE Network

In [11]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [12]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(11,))
x = layers.Reshape((11,1))(encoder_inputs)
x = layers.Conv1D(12,3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1D(24,3,activation="relu", strides=1, padding="same")(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 11, 1)        0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 11, 12)       48          reshape[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 11, 24)       888         conv1d[0][0]                     
____________________________________________________________________________________________

In [13]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(11 * 24, activation="relu")(latent_inputs)
x = layers.Reshape((11, 24))(x)
x = layers.Conv1DTranspose(24, 3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1DTranspose(12, 3, activation="relu", strides=1, padding="same")(x)
x = layers.Conv1DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder_outputs=layers.Reshape((11,))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 264)               792       
_________________________________________________________________
reshape_1 (Reshape)          (None, 11, 24)            0         
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 11, 24)            1752      
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 11, 12)            876       
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 11, 1)             37        
_________________________________________________________________
reshape_2 (Reshape)          (None, 11)                0   

In [14]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 11
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

# Training the VAE Network

In [15]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history=vae.fit(X_train,
        X_train, 
        epochs=20, 
        batch_size=10)

Epoch 1/20
82/82 [==============================] - 0s 4ms/step - loss: 6.5750 - reconstruction_loss: 6.5616 - kl_loss: 0.0134
Epoch 2/20
82/82 [==============================] - 0s 4ms/step - loss: 5.2561 - reconstruction_loss: 5.1778 - kl_loss: 0.0783
Epoch 3/20
82/82 [==============================] - 0s 3ms/step - loss: 4.9161 - reconstruction_loss: 4.5215 - kl_loss: 0.3945
Epoch 4/20
82/82 [==============================] - 0s 6ms/step - loss: 4.8403 - reconstruction_loss: 4.3014 - kl_loss: 0.5389
Epoch 5/20
82/82 [==============================] - 0s 4ms/step - loss: 4.7711 - reconstruction_loss: 4.2230 - kl_loss: 0.5481
Epoch 6/20
82/82 [==============================] - 1s 8ms/step - loss: 4.7643 - reconstruction_loss: 4.1816 - kl_loss: 0.5827
Epoch 7/20
82/82 [==============================] - 0s 4ms/step - loss: 4.7709 - reconstruction_loss: 4.1697 - kl_loss: 0.6011
Epoch 8/20
82/82 [==============================] - 0s 4ms/step - loss: 4.7782 - reconstruction_loss: 4.1519 - 

# Reconstruct and Augment Data

In [17]:
z_mean, z_log_var, z = encoder(X_test)
reconstruction=decoder(z)
X_test=tf.convert_to_tensor(X_test)

fin_X_test=X_test

for i in range(100):
    z_mean, z_log_var, z = encoder(X_test)
    reconstruction=tf.concat([reconstruction,decoder(z)],0)

print(reconstruction)


tf.Tensor(
[[7.38765717e-01 9.53590870e-02 1.73596144e-02 ... 9.64970946e-01
  3.31575066e-05 7.86335440e-05]
 [3.42472792e-01 1.18416280e-01 2.05292910e-01 ... 4.47297126e-01
  2.21391383e-05 1.08427703e-04]
 [4.99680638e-01 2.88451135e-01 1.73330367e-01 ... 4.11593735e-01
  4.00066376e-04 1.48481131e-03]
 ...
 [1.63947701e-01 1.10994786e-01 3.68953764e-01 ... 9.27453637e-02
  1.47803012e-05 3.90023779e-05]
 [2.51345277e-01 3.43365967e-01 4.20610726e-01 ... 1.78999960e-01
  4.69565392e-04 4.50164080e-04]
 [7.14385092e-01 1.13622040e-01 2.34744251e-02 ... 9.04474735e-01
  1.05834595e-04 1.18539749e-04]], shape=(20200, 11), dtype=float32)


In [18]:
reconstruction=minmax.inverse_transform(reconstruction)
reconstruction

array([[ 69.44397736,   2.38143635,   2.08679807, ...,   5.78982568,
        100.00003316,  24.00007863],
       [ 32.19244242,   2.47366512,   3.02646455, ...,   2.68378276,
        100.00002214,  24.00010843],
       [ 46.96998   ,   3.15380454,   2.86665183, ...,   2.46956241,
        100.00040007,  24.00148481],
       ...,
       [ 15.41108394,   2.44397914,   3.84476882, ...,   0.55647218,
        100.00001478,  24.000039  ],
       [ 23.62645602,   3.37346387,   4.10305363, ...,   1.07399976,
        100.00046957,  24.00045016],
       [ 67.15219867,   2.45448816,   2.11737213, ...,   5.42684841,
        100.00010583,  24.00011854]])

In [19]:
reconstruction=np.round(reconstruction)
reconstruction

array([[ 69.,   2.,   2., ...,   6., 100.,  24.],
       [ 32.,   2.,   3., ...,   3., 100.,  24.],
       [ 47.,   3.,   3., ...,   2., 100.,  24.],
       ...,
       [ 15.,   2.,   4., ...,   1., 100.,  24.],
       [ 24.,   3.,   4., ...,   1., 100.,  24.],
       [ 67.,   2.,   2., ...,   5., 100.,  24.]])

In [20]:
reconstruction=pd.DataFrame(reconstruction)

In [21]:
reconstruction.to_csv('reconstruction PUMA==100.csv')